# Generacion de un dataset histologico falso

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from skimage.io import imread
import json
import time
from tqdm import tqdm

In [2]:
with open('configuration.json') as file:
    configuration = json.load(file)

SAVE_PATH = f"{configuration["path"]["data_f"]}/image/"

In [3]:
def backgroud(size):
    loc = random.randint(230, 253)
    return np.ones((size, size, 3)).astype(np.uint8)*loc

def circle(img, center = (255, 255), color = (0, 0, 0), radius = 25):
    new_radius = int(radius*np.random.normal(1, 0.1))
    new_color= (
        random.randint(color[0]-10, color[0]+10),  # Rojo
        random.randint(color[1]-5,  color[1]+5),    # Verde
        random.randint(color[2]-20, color[2]+20)  # Azul
    )
    cv2.circle(img, center, new_radius, new_color, -1)
    return img

def square(img, center = (255, 255), color = (0, 0, 0), radius = 25):
    new_radius = int(radius*np.random.normal(1, 0.1))
    pt = np.ones(2)*np.sqrt(2)*new_radius
    pt1 = (np.array(center)+pt).astype(np.uint8)
    pt2 = (np.array(center)-pt).astype(np.uint8)
    print(center)
    print(pt)
    print(pt1)
    print(pt2)
    new_color= (
        random.randint(color[0]-10, color[0]+10),  # Rojo
        random.randint(color[1]-5,  color[1]+5),    # Verde
        random.randint(color[2]-20, color[2]+20)  # Azul
    )
    cv2.rectangle(img, pt1, pt2, new_color, -1)
    return img

def initial_points(center, radius):
    puntos = []
    for ang in [0, 90, 180, 270]:  # grados
        rad = np.deg2rad(ang)
        new_radius = radius*np.random.normal(1, 0.25)
        x = int(center[0] + new_radius * np.cos(rad))
        y = int(center[1] + new_radius * np.sin(rad))
        puntos.append([x, y])
    return puntos

def add_points(center, points, noise):
    new_points = []
    n = len(points)
    
    for i in range(n):
        p1 = np.array(points[i])
        p2 = np.array(points[(i + 1) % n])
        
        # Radios desde el centro
        r1 = np.linalg.norm(p1 - center)
        r2 = np.linalg.norm(p2 - center)
        rm = (r1 + r2) / 2 * np.random.normal(1, noise)

        # Ángulo medio
        ang1 = np.arctan2(p1[1] - center[1], p1[0] - center[0])
        ang2 = np.arctan2(p2[1] - center[1], p2[0] - center[0])
        if ang2 < ang1:
            ang2 += 2 * np.pi
        ang_medio = (ang1 + ang2) / 2

        # Nuevo punto intermedio
        x = int(center[0] + rm * np.cos(ang_medio))
        y = int(center[1] + rm * np.sin(ang_medio))

        new_points.append(p1.tolist())
        new_points.append([x, y])
    
    return new_points

def poly(img, center, color, radius, steps=2, noise = 0.1):
    points = initial_points(center, radius)
    
    for i in range(steps):
        points = add_points(np.array(center), points, noise/(4**i))
    
    points_array = np.array(points, dtype=np.int32).reshape((-1, 1, 2))
    cv2.fillPoly(img, [points_array], color=color)

    return img
        

In [4]:
def small_color_change(color):
    new_color = (
        random.randint(color[0]-1, color[0]+1),  # Rojo
        random.randint(color[1]-1, color[1]+1),    # Verde 
        random.randint(color[2]-1, color[2]+1)  # Azul
    )
    return new_color

def color_change(color, change = (-20, -10, -30, -20, -10, -5)):
    new_color = (
        random.randint(color[0]+change[0], color[0]+change[1]),  # Rojo
        random.randint(color[1]+change[2], color[1]+change[3]),    # Verde 
        random.randint(color[2]+change[4], color[2]+change[5])  # Azul
    )
    return new_color

def histo():
    size = 2048
    radius = random.randint(15, 25)

    img = backgroud(size)
    centers = []

    color = (
        random.randint(180, 255),
        random.randint(120, 170), 
        random.randint(195, 205)
    )

    for _ in range(random.randint(150, 300)):
        center = (
            random.randint(0, size),
            random.randint(0, size)
        )
        new_color = small_color_change(color)
        img = poly(img, center, new_color, radius*8, steps = 5)
        centers.append(center)   

    color = color_change(color)

    for i in range(len(centers)):
        new_color = small_color_change(color)
        img = poly(img, centers[i], new_color, radius*2, steps = 5)

    color = color_change(color)

    for i in range(len(centers)):
        new_color = small_color_change(color)
        img = poly(img, centers[i], new_color, radius*1.5, steps = 5)

    color = (
        random.randint(80, 120),  # Rojo
        random.randint(0, 30),    # Verde 
        random.randint(100, 150)  # Azul
    )

    for i in range(len(centers)):
        new_color = small_color_change(color)
        img = poly(img, centers[i], new_color, radius, steps = 5)

    # Suavizar para dar apariencia más natural
    img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.resize(img, (1024, 1024))
    img = img[256:768, 256:768]

    return img

def to_grey(img):

    aux = np.mean(img, axis=2).astype(np.uint8)
    grey = np.zeros((512, 512, 3)).astype(np.uint8)
    grey[:, :, 0] = aux
    grey[:, :, 1] = aux
    grey[:, :, 2] = aux

    grey[grey<150] = 10

    grey[grey>180] = 240

    return 255-grey


In [ ]:
length = 300
w = 256
h = 256
r1 = cv2.getRotationMatrix2D((128, 128), 90, 1)
r2 = cv2.getRotationMatrix2D((128, 128), 180, 1)
r3 = cv2.getRotationMatrix2D((128, 128), 270, 1)
start = time.time()
for i in tqdm(range(length)):
    img = histo()
    grey = to_grey(img)

    #Color
    cv2.imwrite(SAVE_PATH+f"{i*32}.png", img[:256, :256])
    cv2.imwrite(SAVE_PATH+f"{i*32+1}.png", cv2.warpAffine(img[:256, :256], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+2}.png", cv2.warpAffine(img[:256, :256], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+3}.png", cv2.warpAffine(img[:256, :256], r3, (w, h)))

    cv2.imwrite(SAVE_PATH+f"{i*32+4}.png", img[:256, 256:])
    cv2.imwrite(SAVE_PATH+f"{i*32+5}.png", cv2.warpAffine(img[:256, 256:], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+6}.png", cv2.warpAffine(img[:256, 256:], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+7}.png", cv2.warpAffine(img[:256, 256:], r3, (w, h)))


    cv2.imwrite(SAVE_PATH+f"{i*32+8}.png", img[256:, :256])
    cv2.imwrite(SAVE_PATH+f"{i*32+9}.png", cv2.warpAffine(img[256:, :256], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+10}.png", cv2.warpAffine(img[256:, :256], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+11}.png", cv2.warpAffine(img[256:, :256], r3, (w, h)))


    cv2.imwrite(SAVE_PATH+f"{i*32+12}.png", img[256:, 256:])
    cv2.imwrite(SAVE_PATH+f"{i*32+13}.png", cv2.warpAffine(img[256:, 256:], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+14}.png", cv2.warpAffine(img[256:, 256:], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+15}.png", cv2.warpAffine(img[256:, 256:], r3, (w, h)))


    #Grey
    cv2.imwrite(SAVE_PATH+f"{i*32+16}.png", grey[:256, :256])
    cv2.imwrite(SAVE_PATH+f"{i*32+17}.png", cv2.warpAffine(grey[:256, :256], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+18}.png", cv2.warpAffine(grey[:256, :256], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+19}.png", cv2.warpAffine(grey[:256, :256], r3, (w, h)))

    cv2.imwrite(SAVE_PATH+f"{i*32+20}.png", grey[:256, 256:])
    cv2.imwrite(SAVE_PATH+f"{i*32+21}.png", cv2.warpAffine(grey[:256, 256:], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+22}.png", cv2.warpAffine(grey[:256, 256:], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+23}.png", cv2.warpAffine(grey[:256, 256:], r3, (w, h)))


    cv2.imwrite(SAVE_PATH+f"{i*32+24}.png", grey[256:, :256])
    cv2.imwrite(SAVE_PATH+f"{i*32+25}.png", cv2.warpAffine(grey[256:, :256], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+26}.png", cv2.warpAffine(grey[256:, :256], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+27}.png", cv2.warpAffine(grey[256:, :256], r3, (w, h)))


    cv2.imwrite(SAVE_PATH+f"{i*32+28}.png", grey[256:, 256:])
    cv2.imwrite(SAVE_PATH+f"{i*32+29}.png", cv2.warpAffine(grey[256:, 256:], r1, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+30}.png", cv2.warpAffine(grey[256:, 256:], r2, (w, h)))
    cv2.imwrite(SAVE_PATH+f"{i*32+30}.png", cv2.warpAffine(grey[256:, 256:], r3, (w, h)))

end = time.time()
elapsed = end-start
print(f"Tiempo de ejecución: {int(elapsed // 3600)} horas, {int((elapsed // 60) % 60)} minutos, {int(elapsed % 60)} segundos")

100%|██████████| 2000/2000 [1:02:26<00:00,  1.87s/it]

Tiempo de ejecución: 1 horas, 2 minutos, 26 segundos
